# HQM Statergy

hqm or high quality momentum statergy 

parts of this code is taken from the free letures on yt (https://www.youtube.com/watch?v=xfzGZB4HhEE&t=10450s)

## Libraries

In [17]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import yfinance as yf
import xlsxwriter

## Funtions

in this example we use snp500, nifty50, dax40, ftse100, cac40.

It scrapes the data from Wikipedia and formats the ticker symbols for use with Yahoo Finance.

The script also includes a function to get the company names from a specified URL and table.

The company names are formatted to include a specified suffix and limited to a specified length.

In [18]:
def Compony_Names(url, tableno, coloumn, length, fix):
    url = url
    data_table = pd.read_html(url)
    name = data_table[tableno][coloumn].tolist()
    for i in range(len(name)):
        name[i] = name[i] + fix
    name = name[:length]
    return name

def price_data(componyList, start, end, currency):
    data = yf.download(componyList, start=start, end=end)['Close'].values
    currency_data = yf.download(currency, start=start, end=end)['Close'].values
    fianal_data = data * currency_data
    return fianal_data

def X_Time_return(componylist, time):
    try:
        stock_data = yf.download(componylist, period=time, interval='1d')
        stock_data['Daily Return'] = stock_data['Close'].pct_change()
        six_month_return = (1 + stock_data['Daily Return']).prod() - 1
        return six_month_return
    except Exception as e:
        print(f"Error fetching data for {componylist}: {e}")
        return None

In [20]:
us_compony = Compony_Names('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', 0, 'Symbol', 50, '')
in_compony = Compony_Names('https://en.wikipedia.org/wiki/NIFTY_50', 1, 'Symbol', 50, '.NS')
ge_compony = Compony_Names('https://en.wikipedia.org/wiki/DAX', 4, 'Ticker', 40, '')
uk_compony = Compony_Names('https://en.wikipedia.org/wiki/FTSE_100_Index', 4, 'Ticker', 50, '.L')
fr_compony = Compony_Names('https://en.wikipedia.org/wiki/CAC_40', 4, 'Ticker', 40, '')

In [21]:
all_compony = us_compony + in_compony + ge_compony + uk_compony + fr_compony

# some correction related to perticular compnies

all_compony.remove('TCS.NS')
ge_compony.remove('AIR.PA')

# creating a set to remove duplicates

unique_compony = set(all_compony)
unique_compony = list(unique_compony)

In [22]:
print(unique_compony)

['MARUTI.NS', 'ABNB', 'AOS', 'ADANIENT.NS', 'AXISBANK.NS', 'AV.L', 'MC.PA', 'MO', 'ADBE', 'ACN', 'DB1.DE', 'QIA.DE', 'CBK.DE', 'DRREDDY.NS', 'DCC.L', 'SHRIRAMFIN.NS', 'TECHM.NS', 'BP.L', 'TEP.PA', 'HLMA.L', 'POWERGRID.NS', 'APOLLOHOSP.NS', 'ADM', 'BAS.DE', 'RWE.DE', 'ICICIBANK.NS', 'SAF.PA', 'MRK.DE', 'HINDALCO.NS', 'CS.PA', 'CRDA.L', 'LT.NS', 'IMI.L', 'ADI', 'EN.PA', 'BAJFINANCE.NS', 'JSWSTEEL.NS', 'ENGI.PA', 'DGE.L', 'HEN3.DE', 'BA.L', 'AME', 'ABF.L', 'ALL', 'ML.PA', 'EDV.L', 'WIPRO.NS', 'BATS.L', 'RELIANCE.NS', 'T', 'SAN.PA', 'RHM.DE', 'HSBA.L', 'SU.PA', 'SRT3.DE', 'ADANIPORTS.NS', 'HNR1.DE', 'SIE.DE', 'EICHERMOT.NS', 'AMAT', 'AAL.L', 'ENT.L', 'SY1.DE', 'RI.PA', 'AMT', 'ALW.L', 'HLN.L', 'M&M.NS', 'BKG.L', 'STLAP.PA', 'AMZN', 'AKAM', 'ANTO.L', 'GRASIM.NS', 'ALLE', 'ALGN', 'BEL.NS', 'EDEN.PA', 'CIPLA.NS', 'AAPL', 'CPG.L', 'MT.AS', 'BN.PA', 'AMCR', 'AIG', 'ASIANPAINT.NS', 'LNT', 'GLEN.L', 'NTPC.NS', 'ALV.DE', 'EL.PA', 'COALINDIA.NS', 'HEROMOTOCO.NS', 'CAP.PA', 'PAH3.DE', 'NESTLEIND.NS'

In [25]:
hqm_columns = [
    'Stock name',
    'Close price',
    'Number of Shares to Buy',
    'One year price return',
    'One year return percetile',
    'Six month price return',
    'Six month return percetile',
    'Three month price return',
    'Three month return percetile',
    'One month price return',
    'One month return percetile',
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for noob in unique_compony:
    stock_data = yf.Ticker(noob)
    one_year_returns = X_Time_return(noob, '1y')
    six_month_returns = X_Time_return(noob, '6mo')
    three_month_returns = X_Time_return(noob, '3mo')
    one_month_returns = X_Time_return(noob, '1mo')
    hqm_dataframe = pd.concat([
        hqm_dataframe, 
        pd.DataFrame({
            'Stock name': [noob],
            'One year price return': [one_year_returns],
            'Six month price return': [six_month_returns],
            'Three month price return': [three_month_returns],
            'One month price return': [one_month_returns],

        })
    ], ignore_index=True)

hqm_dataframe

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ICG.L']: YFPricesMissingError('possibly delisted; no price data found  (period=6mo)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\garvg\AppData\Local\Temp\ipykernel_35008\3518327202.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = pd.concat([
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[****************

KeyboardInterrupt: 

okay so im kinda confused on this part so if you have any seggestions please do so. for now its just a bunch of elif statement

In [8]:
results = []  # Create an empty list to store results

for t in unique_compony:
    if t in us_compony:
        result = price_data(t, '2025-04-02', '2025-04-03', 'INR=X')
    elif t in ge_compony:
        result = price_data(t, '2025-04-02', '2025-04-03', 'EURINR=X')
    elif t in uk_compony:
        result = price_data(t, '2025-04-02', '2025-04-03', 'GBPINR=X')
    elif t in fr_compony:
        result = price_data(t, '2025-04-02', '2025-04-03', 'EURINR=X')
    elif t in in_compony:
        result = yf.download(t, start='2025-04-02', end='2025-04-03')['Close'].values
    else:
        result = t
        print(t)
    if isinstance(result, np.ndarray):
        results.append(result[0][0])  # Flatten the array and add the value
    else:
        results.append(result)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
hqm_dataframe['Close price'] = results
hqm_dataframe

,Stock name,Close price,Number of Shares to Buy,One year price return,One year return percetile,Six month price return,Six month return percetile,Three month price return,Three month return percetile,One month price return,One month return percetile
0,MARUTI.NS,11716.099609,NaN,-0.077726,NaN,-0.089146,NaN,-0.023278,NaN,-0.011557,NaN
1,ABNB,10512.920516,NaN,-0.340669,NaN,-0.180987,NaN,-0.211095,NaN,-0.251929,NaN
2,AOS,5660.540033,NaN,-0.279611,NaN,-0.290017,NaN,-0.095723,NaN,-0.067820,NaN
3,ADANIENT.NS,2369.399902,NaN,-0.272583,NaN,-0.249466,NaN,-0.056497,NaN,0.088516,NaN
4,AXISBANK.NS,1084.449951,NaN,0.027463,NaN,-0.073914,NaN,0.026237,NaN,0.089339,NaN
...,...,...,...,...,...,...,...,...,...,...,...
223,CNA.L,16665.682010,NaN,0.163162,NaN,0.247885,NaN,0.078216,NaN,0.001698,NaN
224,INFY.NS,1550.150024,NaN,0.001575,NaN,-0.234586,NaN,-0.250897,NaN,-0.140171,NaN
225,HSX.L,132418.335045,NaN,-0.076249,NaN,-0.023726,NaN,0.009083,NaN,-0.052048,NaN
226,HEI.DE,15217.797645,NaN,0.572875,NaN,0.555761,NaN,0.235222,NaN,0.068406,NaN


# percentile calculation

In [10]:
time_periods = ['One year', 'Six month', 'Three month', 'One month']

for row in hqm_dataframe.index:
    for period in time_periods:
        change_col = f'{period} price return'
        percentile_col = f'{period} return percetile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Stock name,Close price,Number of Shares to Buy,One year price return,One year return percetile,Six month price return,Six month return percetile,Three month price return,Three month return percetile,One month price return,One month return percetile
0,MARUTI.NS,11716.099609,NaN,-0.077726,0.385965,-0.089146,0.491228,-0.023278,0.548246,-0.011557,0.657895
1,ABNB,10512.920516,NaN,-0.340669,0.109649,-0.180987,0.254386,-0.211095,0.127193,-0.251929,0.026316
2,AOS,5660.540033,NaN,-0.279611,0.162281,-0.290017,0.100877,-0.095723,0.355263,-0.067820,0.491228
3,ADANIENT.NS,2369.399902,NaN,-0.272583,0.166667,-0.249466,0.144737,-0.056497,0.464912,0.088516,0.925439
4,AXISBANK.NS,1084.449951,NaN,0.027463,0.557018,-0.073914,0.52193,0.026237,0.666667,0.089339,0.929825
...,...,...,...,...,...,...,...,...,...,...,...
223,CNA.L,16665.682010,NaN,0.163162,0.75,0.247885,0.942982,0.078216,0.754386,0.001698,0.723684
224,INFY.NS,1550.150024,NaN,0.001575,0.504386,-0.234586,0.179825,-0.250897,0.074561,-0.140171,0.263158
225,HSX.L,132418.335045,NaN,-0.076249,0.390351,-0.023726,0.618421,0.009083,0.631579,-0.052048,0.530702
226,HEI.DE,15217.797645,NaN,0.572875,0.97807,0.555761,0.986842,0.235222,0.951754,0.068406,0.894737


# HQM score calculation

bascically the mean of iy, 6mo, 3mo, 1mo price return 

In [11]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} return percetile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)

hqm_dataframe

,Stock name,Close price,Number of Shares to Buy,One year price return,One year return percetile,Six month price return,Six month return percetile,Three month price return,Three month return percetile,One month price return,One month return percetile,HQM Score
0,MARUTI.NS,11716.099609,NaN,-0.077726,0.385965,-0.089146,0.491228,-0.023278,0.548246,-0.011557,0.657895,0.520833
1,ABNB,10512.920516,NaN,-0.340669,0.109649,-0.180987,0.254386,-0.211095,0.127193,-0.251929,0.026316,0.129386
2,AOS,5660.540033,NaN,-0.279611,0.162281,-0.290017,0.100877,-0.095723,0.355263,-0.067820,0.491228,0.277412
3,ADANIENT.NS,2369.399902,NaN,-0.272583,0.166667,-0.249466,0.144737,-0.056497,0.464912,0.088516,0.925439,0.425439
4,AXISBANK.NS,1084.449951,NaN,0.027463,0.557018,-0.073914,0.52193,0.026237,0.666667,0.089339,0.929825,0.668860
...,...,...,...,...,...,...,...,...,...,...,...,...
223,CNA.L,16665.682010,NaN,0.163162,0.75,0.247885,0.942982,0.078216,0.754386,0.001698,0.723684,0.792763
224,INFY.NS,1550.150024,NaN,0.001575,0.504386,-0.234586,0.179825,-0.250897,0.074561,-0.140171,0.263158,0.255482
225,HSX.L,132418.335045,NaN,-0.076249,0.390351,-0.023726,0.618421,0.009083,0.631579,-0.052048,0.530702,0.542763
226,HEI.DE,15217.797645,NaN,0.572875,0.97807,0.555761,0.986842,0.235222,0.951754,0.068406,0.894737,0.952851


now here you can select any no of stocks that you want. for now i will go with 50 stocks with the highest hqm score

In [12]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True ,inplace=True)
hqm_dataframe

,Stock name,Close price,Number of Shares to Buy,One year price return,One year return percetile,Six month price return,Six month return percetile,Three month price return,Three month return percetile,One month price return,One month return percetile,HQM Score
0,RHM.DE,119766.281433,NaN,1.402417,0.991228,1.453846,0.995614,1.104222,1.0,0.137255,0.991228,0.994518
1,HEI.DE,15217.797645,NaN,0.572875,0.97807,0.555761,0.986842,0.235222,0.951754,0.068406,0.894737,0.952851
2,HO.PA,22743.589424,NaN,0.539416,0.964912,0.627570,0.991228,0.727042,0.995614,0.043421,0.815789,0.941886
3,FRES.L,103379.226482,NaN,0.521965,0.960526,0.293239,0.95614,0.277174,0.97807,0.054487,0.846491,0.935307
4,AAF.L,18540.780085,NaN,0.479128,0.942982,0.361818,0.969298,0.257983,0.960526,0.059448,0.864035,0.934211
5,CCH.L,392055.621887,NaN,0.509902,0.947368,0.299114,0.960526,0.281136,0.982456,0.023269,0.785088,0.918860
6,T,2409.068156,NaN,0.618194,0.986842,0.254569,0.947368,0.193723,0.929825,0.017571,0.77193,0.908991
7,KOTAKBANK.NS,2154.600098,NaN,0.220185,0.798246,0.178524,0.899123,0.197960,0.934211,0.117491,0.97807,0.902412
8,ENGI.PA,1702.306349,NaN,0.297490,0.855263,0.196498,0.912281,0.179291,0.916667,0.071118,0.899123,0.895833
9,DB1.DE,25550.756773,NaN,0.429052,0.929825,0.244381,0.938596,0.181117,0.921053,0.020392,0.780702,0.892544


# No of stocks to buy caluculation

In [15]:
porfolio_size = input('how much money do you have?: ')

try:
    paisa = float(porfolio_size)

except ValueError:  
    print("please inter an integer value \ntry again")
    porfolio_size = input('how much money do you have?: ')
    paisa = float(porfolio_size)
    

In [16]:
position_size = paisa / len(hqm_dataframe.index)
for i in range(0,len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Close price'])
    
hqm_dataframe

,Stock name,Close price,Number of Shares to Buy,One year price return,One year return percetile,Six month price return,Six month return percetile,Three month price return,Three month return percetile,One month price return,One month return percetile,HQM Score
0,RHM.DE,119766.281433,8,1.402417,0.991228,1.453846,0.995614,1.104222,1.0,0.137255,0.991228,0.994518
1,HEI.DE,15217.797645,65,0.572875,0.97807,0.555761,0.986842,0.235222,0.951754,0.068406,0.894737,0.952851
2,HO.PA,22743.589424,43,0.539416,0.964912,0.627570,0.991228,0.727042,0.995614,0.043421,0.815789,0.941886
3,FRES.L,103379.226482,9,0.521965,0.960526,0.293239,0.95614,0.277174,0.97807,0.054487,0.846491,0.935307
4,AAF.L,18540.780085,53,0.479128,0.942982,0.361818,0.969298,0.257983,0.960526,0.059448,0.864035,0.934211
5,CCH.L,392055.621887,2,0.509902,0.947368,0.299114,0.960526,0.281136,0.982456,0.023269,0.785088,0.918860
6,T,2409.068156,415,0.618194,0.986842,0.254569,0.947368,0.193723,0.929825,0.017571,0.77193,0.908991
7,KOTAKBANK.NS,2154.600098,464,0.220185,0.798246,0.178524,0.899123,0.197960,0.934211,0.117491,0.97807,0.902412
8,ENGI.PA,1702.306349,587,0.297490,0.855263,0.196498,0.912281,0.179291,0.916667,0.071118,0.899123,0.895833
9,DB1.DE,25550.756773,39,0.429052,0.929825,0.244381,0.938596,0.181117,0.921053,0.020392,0.780702,0.892544
